# Power Outages
This project uses major power outage data in the continental U.S. from January 2000 to July 2016. Here, a major power  outage is defined as a power outage that impacted at least 50,000 customers or caused an unplanned firm load loss of atleast 300MW. Interesting questions to consider include:
- Where and when do major power outages tend to occur?
- What are the characteristics of major power outages with higher severity? Variables to consider include location, time, climate, land-use characteristics, electricity consumption patterns, economic characteristics, etc. What risk factors may an energy company want to look into when predicting the location and severity of its next major power outage?
- What characteristics are associated with each category of cause?
- How have characteristics of major power outages changed over time? Is there a clear trend?

### Getting the Data
The data is downloadable [here](https://engineering.purdue.edu/LASCI/research-data/outages/outagerisks).

A data dictionary is available at this [article](https://www.sciencedirect.com/science/article/pii/S2352340918307182) under *Table 1. Variable descriptions*.

### Cleaning and EDA
- Note that the data is given as an Excel file rather than a CSV. Open the data in Excel or another spreadsheet application and determine which rows and columns of the Excel spreadsheet should be ignored when loading the data in pandas.
- Clean the data.
    - The power outage start date and time is given by `OUTAGE.START.DATE` and `OUTAGE.START.TIME`. It would be preferable if these two columns were combined into one datetime column. Combine `OUTAGE.START.DATE` and `OUTAGE.START.TIME` into a new datetime column called `OUTAGE.START`. Similarly, combine `OUTAGE.RESTORATION.DATE` and `OUTAGE.RESTORATION.TIME` into a new datetime column called `OUTAGE.RESTORATION`.
- Understand the data in ways relevant to your question using univariate and bivariate analysis of the data as well as aggregations.

*Hint 1: pandas can load multiple filetypes: `pd.read_csv`, `pd.read_excel`, `pd.read_html`, `pd.read_json`, etc.*

*Hint 2: `pd.to_datetime` and `pd.to_timedelta` will be useful here.*

*Tip: To visualize geospatial data, consider [Folium](https://python-visualization.github.io/folium/) or another geospatial plotting library.*

### Assessment of Missingness
- Assess the missingness of a column that is not missing by  design.

### Hypothesis Test
Find a hypothesis test to perform. You can use the questions at the top of the notebook for inspiration.

# Summary of Findings

### Introduction
TODO

### Cleaning and EDA
TODO

### Assessment of Missingness
TODO

### Hypothesis Test
TODO

# Code

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

### Cleaning and EDA

In [152]:
# Load data
cols = (['OBS', 'YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
         'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME',
         'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED']) # NERC.REGION, HURRICANE.NAMES
fp = os.path.join('data', 'outage.xlsx')
df = pd.read_excel(fp, header=0, skiprows=[0, 1, 2, 3, 4, 6])[cols] # Load dataframe for my use, skip unuseful rows
df.head()

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED
0,1,2011,7.0,Minnesota,MN,East North Central,-0.3,normal,2011-07-01,17:00:00,2011-07-03,20:00:00,severe weather,NaN,3060.0,NaN,70000.0
1,2,2014,5.0,Minnesota,MN,East North Central,-0.1,normal,2014-05-11,18:38:00,2014-05-11,18:39:00,intentional attack,vandalism,1.0,NaN,NaN
2,3,2010,10.0,Minnesota,MN,East North Central,-1.5,cold,2010-10-26,20:00:00,2010-10-28,22:00:00,severe weather,heavy wind,3000.0,NaN,70000.0
3,4,2012,6.0,Minnesota,MN,East North Central,-0.1,normal,2012-06-19,04:30:00,2012-06-20,23:00:00,severe weather,thunderstorm,2550.0,NaN,68200.0
4,5,2015,7.0,Minnesota,MN,East North Central,1.2,warm,2015-07-18,02:00:00,2015-07-19,07:00:00,severe weather,NaN,1740.0,250.0,250000.0


In [153]:
# Combine date and time into datetime
df['OUTAGE.START'] = (df['OUTAGE.START.DATE'] + 
                      pd.to_timedelta(df['OUTAGE.START.TIME'].astype(str))) # Combine START into date
df['OUTAGE.RESTORATION'] = (df['OUTAGE.RESTORATION.DATE'] +
                            pd.to_timedelta(df['OUTAGE.RESTORATION.TIME'].astype(str))) # Combine RESTORATION into date
outage = df.drop(columns=['OUTAGE.START.DATE', 'OUTAGE.START.TIME',
                          'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME']) # Drop columns
outage.head()

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,OUTAGE.START,OUTAGE.RESTORATION
0,1,2011,7.0,Minnesota,MN,East North Central,-0.3,normal,severe weather,NaN,3060.0,NaN,70000.0,2011-07-01 17:00:00,2011-07-03 20:00:00
1,2,2014,5.0,Minnesota,MN,East North Central,-0.1,normal,intentional attack,vandalism,1.0,NaN,NaN,2014-05-11 18:38:00,2014-05-11 18:39:00
2,3,2010,10.0,Minnesota,MN,East North Central,-1.5,cold,severe weather,heavy wind,3000.0,NaN,70000.0,2010-10-26 20:00:00,2010-10-28 22:00:00
3,4,2012,6.0,Minnesota,MN,East North Central,-0.1,normal,severe weather,thunderstorm,2550.0,NaN,68200.0,2012-06-19 04:30:00,2012-06-20 23:00:00
4,5,2015,7.0,Minnesota,MN,East North Central,1.2,warm,severe weather,NaN,1740.0,250.0,250000.0,2015-07-18 02:00:00,2015-07-19 07:00:00


### Assessment of Missingness

In [ ]:
# TODO

### Hypothesis Test

In [ ]:
# TODO